In [27]:
import os
import re
import json
import tqdm
import torch
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import ray
import openai
import ast
import time
import pickle

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

torch.backends.cuda.matmul.allow_tf32 = True

from transformers import LlamaTokenizer, LlamaForCausalLM

In [2]:
class RedundantParenthesesRemover(ast.NodeTransformer):
    def visit_Expr(self, node):
        self.generic_visit(node)
        if isinstance(node.value, ast.BinOp):
            return node.value
        return node

def remove_redundant_parentheses(expression):
    # Parse the expression
    parsed_expression = ast.parse(expression)

    # Remove redundant parentheses
    transformer = RedundantParenthesesRemover()
    transformed_expression = transformer.visit(parsed_expression)

    # Convert the transformed expression back to a string
    simplified_expression = ast.unparse(transformed_expression)

    return simplified_expression
    
def divide(a, b):
    return f"({a} / {b})"

def subtract(a, b):
    return f"({a} - {b})"

def multiply(a, b):
    return f"({a} * {b})"

def add(a, b):
    return f"({a} + {b})"

def exp(a, b):
    return f"({a} ** {b})"

def greater(a, b):
    return f"({a} > {b})"

def translate_expr(expr):
    if "table" in expr:
        return expr
    
    # replace const_m1
    expr = re.sub(r'const_m1', r'-1', expr)

    # change % to / 100
    expr = re.sub(r'([0-9]*\.?[0-9]+)%', r'divide(\1 , 100)', expr)
    
    expr = re.sub(r'const_([0-9]*\.?[0-9]+)', r'\1', expr)
    try:
        new_expr = eval(expr)
        new_expr = remove_redundant_parentheses(new_expr)
    except Exception as e:
        print(e, expr)
        new_expr = expr
    
    return new_expr

def convert_to_markdown(data):
    markdown = "|"
    
    # Add table headers
    for header in data[0]:
        markdown += header + "|"
    markdown += "\n|"
    
    # Add table header separators
    for _ in data[0]:
        markdown += "---|"
    markdown += "\n"
    
    # Add table rows
    for row in data[1:]:
        markdown += "|"
        for cell in row:
            markdown += cell + "|"
        markdown += "\n"
        
    return markdown

def extract_answer(response):
    # extract content inside Calculate()
    matches = re.findall(r"Calculate\(([\(\)0-9 ><,\.\/\+\-\*]*)\)", response)
    if len(matches) == 0:
        if "Yes" in response:
            return "Yes"
        elif "No" in response:
            return "No"
        else:
            return ""
    else:
        output = matches[0].replace(",", "")
        return output
    
def if_exec_correct(t_prog, g_prog):
    try:
        t_exec = eval(t_prog)

        if type(t_exec) == bool and g_prog in ["Yes", "No"]:
            t_exec = "Yes" if t_exec else "No"

            if t_exec == g_prog:
                return True
            
        g_exec = eval(g_prog)

        if t_exec == g_exec:
            return True
        elif t_exec * 100 == g_exec:
            return True
        elif t_exec * 100 == -g_exec:
            return True
        elif t_exec == g_exec * 100:
            return True
        elif t_exec == -g_exec * 100:
            return True
        elif t_exec * 1000000 == g_exec:
            return True
        elif t_exec * 1000000 == -g_exec:
            return True
        elif t_exec == g_exec * 1000000:
            return True
        elif t_exec == -g_exec * 1000000:
            return True
        elif t_exec == -g_exec:
            return True
    except:
        return False

    return False

In [3]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [4]:
# state_dict = torch.load("./llama-65b/checkpoint-500/hf_llama.pth", map_location="cpu")
# model = LlamaForCausalLM.from_pretrained("decapoda-research/llama-65b-hf", device_map="auto", torch_dtype=torch.bfloat16)
# model.load_state_dict(state_dict)

In [5]:
system_prompt = (
                "You need to answer the user's question in the ### Question ### section.\n" \
                "You need to provide the answer in the format 'Calculate(a + b)', where the expression needs to be python excutable." \
                # "You can calculate the average of a column by using the function 'Average(table_column_name)'.\n" \
                # "Similarly, you can calculate the sum, the maximum, the minimum, the count of a column by using the functions "\
                # "'Sum(table_column_name)', 'Max(table_column_name)', 'Min(table_column_name)', 'Count(table_column_name)' respectively.\n" \
                # "You only use the table's column name inside those operations\n" \
                "For example, if the question is 'What is the sum of 1 + 2?', you need to answer 'Calc(1 + 2)'." \
                "if the question is 'Is 123 greater than 231?', you need to answer 'Calc(123 > 231)'." \
                # "|Age|\n|---|\n|12|\n|15|\n|16|\n\n What is the average age? The answer is 'Calculate(Average(Age))'" \
                "DO NOT give anything else other than'Calculate()'."
                )

In [6]:
filepath = "../FinQA/dataset/test.json"

with open(filepath) as f:
    data = json.load(f)

programs = []
translated_programs = []
answers = []

for item in tqdm.tqdm(data):
    table_md = convert_to_markdown(item["table_ori"])
    question = item["qa"]["question"]
    
    pre_text = "\n".join(item["pre_text"])
    post_text = "\n".join(item["post_text"])

    programs.append(item["qa"]["program_re"])
    translated_programs.append(translate_expr(item["qa"]["program_re"]))
    answers.append(item["qa"]["answer"])

100%|██████████| 1147/1147 [00:00<00:00, 14019.56it/s]


In [7]:
# with open("./results/llama65b-finetune-300.json") as f:
#     results = json.load(f)

In [18]:
responses = results["responses"]

In [12]:
gen_programs = []

for response in responses:
    gen_programs.append(extract_answer(response))

In [13]:
# calculate accuracy
prog_correct = 0
exec_correct = 0

wrong_indices = []

for i in range(len(translated_programs)):
    t_prog = translated_programs[i].replace(" ", "")
    g_prog = gen_programs[i].replace(" ", "")

    if t_prog == g_prog:
        prog_correct += 1

    if if_exec_correct(t_prog, g_prog):
        exec_correct += 1
    else:
        wrong_indices.append(i)
        print(i, t_prog, "|", g_prog)

0 5829-5735 | 5829-5735)/5735)*100)/100)*1000000)*1000)00)*1000)00)*1000)00)*100)/100)*1000000)*1000000)*1000000)*1000000
1 8.1/56.0 | 8.1/56.0)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100
2 (153.7-139.9)/139.9 | (20692-139.9)/139.9)/139.9)/100)/100)/100)/100)/100)/100)/100)/100)/100)/100)/100)/100)/100)/100)/100
3 121.4/4187.8 | 121.4/4187.8)*100)/100)*100)/100)*100)/100)*100)/100)*100)/100)*100)/100)*100)/100)*100)/100)*100)/100
4 (318.46-100)/100-(206.49-100)/100 | (318.46-100)/100-(206.49-100)/100)/2)-(250.0-100)/100)/2)-(350.0-100)/100)/2)-(250.0-100)/100)/2)-(350.0-100
5 (92710000-86842000)/86842000 | (100690000-92710000)/92710000)/92710000)/92710000)/92710000)/92710000)/92710000)/92710000)/92710000)/92710000
6 463/4612 | 463/4612)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100
7 301/2575 | 301/2575)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*100)*

In [14]:
exec_correct / len(translated_programs), prog_correct / len(translated_programs)

(0.12205754141238012, 0.11333914559721012)

In [8]:
model = LlamaForCausalLM.from_pretrained("decapoda-research/llama-7b-hf", device_map="auto", torch_dtype=torch.bfloat16)

model_cpu = LlamaForCausalLM.from_pretrained("./llama-7b/checkpoint-1125", torch_dtype=torch.bfloat16)

model.load_state_dict(model_cpu.state_dict())

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

<All keys matched successfully>

In [9]:
# %%
tokenizer.pad_token_id = tokenizer.unk_token_id

# %%
all_input_ids = []

for item in tqdm.tqdm(data):
    table_md = convert_to_markdown(item["table_ori"])
    question = item["qa"]["question"]
    
    pre_text = "\n".join(item["pre_text"])
    post_text = "\n".join(item["post_text"])
    
    context = f"{pre_text}\n\n{table_md}\n\n{post_text}\n\n"
    user_prompt = f"### Context ###\n\n{context}### Question ###\n\n{question}"
    input_prompt = f"### Instruction ###\n\n{system_prompt}\n\n{user_prompt}\n\n### Answer ###\n\n"

    input_ids = tokenizer.encode(input_prompt, add_special_tokens=False)

    input_ids_length = len(input_ids) 
    max_seq_length = 1984 
    while input_ids_length > max_seq_length:
        # truncate the first input_ids_length - max_seq_length tokens
        context = context.split(" ")[input_ids_length - max_seq_length:]
        context = " ".join(context)
        # recreate the input_text
        user_prompt = f"### Context ###\n\n{context}### Question ###\n\n{question}"
        input_prompt = f"### Instruction ###\n\n{system_prompt}\n\n{user_prompt}\n\n### Answer ###\n\n"

        input_ids = tokenizer.encode(input_prompt, add_special_tokens=False)

        input_ids_length = len(input_ids)

    input_ids = torch.tensor(input_ids)

    all_input_ids.append(input_ids)

100%|██████████| 1147/1147 [00:04<00:00, 239.94it/s]


In [22]:
# %%
batch_size = 1

responses = []

for i in tqdm.tqdm(range(0, len(all_input_ids), batch_size)):
    batch_input_ids = all_input_ids[i:i+batch_size]
    batch_input_ids = [item.flip(0) for item in batch_input_ids]
    batch_input_ids = pad_sequence(batch_input_ids, 
                                   batch_first=True,
                                   padding_value=tokenizer.pad_token_id).to(model.device)
    batch_input_ids = batch_input_ids.flip(1)

    attention_mask = (batch_input_ids != tokenizer.pad_token_id).bool().to(model.device)

    output = model.generate(batch_input_ids,
                            attention_mask=attention_mask,
                            do_sample=True,
                            num_beams=4,
                            top_p=0.9, 
                            temperature=0.9, 
                            max_new_tokens=128, 
                            eos_token_id=13, 
                            pad_token_id=tokenizer.eos_token_id, 
                            early_stopping=True)

    for j in range(output.shape[0]):
        input_ids_length = batch_input_ids.shape[1]
        response = tokenizer.decode(output[j][input_ids_length:], skip_special_tokens=True)

        responses.append(response)

  0%|          | 1/1147 [00:09<3:01:34,  9.51s/it]


KeyboardInterrupt: 

In [24]:
responses[0]

' Calculate(5829 - 5735) / 5735) * 100) / 100) * 1000000) * 100) / 100) * 1000000) * 100) / 100) * 1000000) * 100) / 100) * 1000000) * 100) / 100) * 1000'

In [11]:
result = {
            "responses": responses,
            "model_name": "llama-7b-finetune-1125",
          }

with open("./results/llama7b-finetune-1125.json", "w") as f:
    json.dump(result, f, indent=4)


In [28]:
# check llama_funetune pkl

with open("./llama_finetune_dataset.pkl", "rb") as f:
    dataset = pickle.load(f)

In [31]:
tokenizer.decode(dataset[0]['input_ids'])

"### Instruction ###\n\nYou need to answer the user's question in the ### Question ### section.\nYou need to provide the answer in the format 'Calculate(a + b)', where the expression needs to be python excutable.For example, if the question is 'What is the sum of 1 + 2?', you need to answer 'Calculate(1 + 2)'.if the question is 'Is 123 greater than 231?', you need to answer 'Calculate(123 > 231)'.DO NOT give anything else other than'Calculate()'.\n\n### Context ###\n\ninterest rate to a variable interest rate based on the three-month libor plus 2.05% ( 2.05 % ) ( 2.34% ( 2.34 % ) as of october 31 , 2009 ) .\nif libor changes by 100 basis points , our annual interest expense would change by $ 3.8 million .\nforeign currency exposure as more fully described in note 2i .\nin the notes to consolidated financial statements contained in item 8 of this annual report on form 10-k , we regularly hedge our non-u.s .\ndollar-based exposures by entering into forward foreign currency exchange contr